In [68]:
import pandas as pd
import numpy as np
import scipy.optimize as optimize



In [69]:
par = 1000


In [70]:



def bond_ytm(price, par, T, coup, freq=2, guess=0.05):
    freq = float(freq)
    periods = T*freq
    coupon = coup/100.*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    ytm_func = lambda y: sum([coupon/(1+y/freq)**(freq*t) for t in dt]) + par/(1+y/freq)**(freq*T) - price
        
        
        
    return optimize.newton(ytm_func, guess)


ytm = bond_ytm(1000, 1000, 10, 20, 2)
ytm

0.19999999999999996

In [71]:
def bond_price(par, T, ytm, coup, freq=2):
    freq = float(freq)
    periods = T*freq
    coupon = (coup/100)*par/freq
    dt = [(i+1)/freq for i in range(int(periods))]
    price = sum([coupon/(1+ytm/freq)**(freq*t) for t in dt]) + \
            par/(1+ytm/freq)**(freq*T)
    return price


pv = bond_price(1000, 10, 0.15, 15)
pv

1000.0000000000003

In [72]:
def bond_fv(price, T, ytm, coup, freq=2):
    periods = T * freq
    coupon_payment = coup / freq
    fv = 0

    for i in range(1, periods + 1):
        fv = coupon_payment / ((1 + ytm**0.5) ** i)
    vf = fv+1000

   

    return vf

bond_fv(5, 5, 0.1, 20)

1000.6407513344584

In [73]:
def durations_explicit(c, ytm, t, n):
    """Parameters:
        c = coupon rate per period
        y = yield per period
        m = periods per year
        n = periods remaining"""
    if c==y: # Shorter explicit formula if coupon rate per period = yield per period
        macaulay_duration = ((1+y)/(m*y))*(1 - (1 / (1+y)**n))
        
        
       
        return macaulay_duration
    else:
        macaulay_duration = ((1+y) / (m*y)) - ( (1 + y + n*(c-y)) / ((m*c* ((1+y)**n - 1)) + m*y) )
       
       
       
        return macaulay_duration


In [74]:
def effective_duration(price, par, T, coup, freq=2, dy=0.01):
    ytm = bond_ytm(price, par, T, coup)
    
    ytm_minus = ytm - dy   
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    mduration = (price_minus-price_plus)/(2*price*dy)
    return mduration



In [75]:
def bond_convexity(price, par, T, coup, freq=2, dy=0.01):
    ytm = bond_ytm(price, par, T, coup, freq)

    ytm_minus = ytm - dy    
    price_minus = bond_price(par, T, ytm_minus, coup, freq)
    
    ytm_plus = ytm + dy
    price_plus = bond_price(par, T, ytm_plus, coup, freq)
    
    convexity = (price_minus+price_plus-2*price)/(price*dy**2)
    return convexity

bond_convexity(1000, 1000, 10, 50, 2)


7.541082988354901

In [76]:
#Sistema para gerar titulos 
cdi_gen = np.random.uniform(0, 0.20)

cdif = [cdi_gen]

for i in range(9):
    cdi_gen = cdi_gen * (1 + np.random.uniform(-0.15, 0.15))
    cdif.append(cdi_gen)


t_anos = 10

anos = [pd.Timestamp(year=i+2024, month=1, day=1) for i in range(t_anos)]


cdi = pd.DataFrame({"CDI Futuro" : cdif, "Ano" : anos})

cdi['Ano'] = pd.to_datetime(cdi['Ano'])

cdi['Ano'] = cdi['Ano'].dt.year.astype(str)

cdi.set_index("Ano")

    
    

,CDI Futuro
Ano,
2024,0.063925
2025,0.072652
2026,0.080167
2027,0.075822
2028,0.079016
2029,0.089307
2030,0.094262
2031,0.080982
2032,0.075784


In [77]:
class bonds:
    def __init__(self):
        self.par_value = 1000
        self.coupon = np.random.randint(low=0, high= self.par_value*0.05)
        self.maturity = np.random.randint(2024, 2033)
        self.vencimento = self.maturity - 2023
        ano_maturidade = str(self.maturity)
        
        
        
        
        for index, row in cdi.iterrows():
            if row['Ano'] == ano_maturidade:
                self.ytm_init = row['CDI Futuro']
                break
        
        self.pv = bond_price(self.par_value, self.vencimento,self.ytm_init , self.coupon)
        self.efecdu = effective_duration(self.pv, self.par_value, self.vencimento, self.coupon)
        self.convex = bond_convexity(self.pv, self.par_value,self.vencimento ,self.coupon )
    def risco_buy(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0, high= 0.01)
        elif teste == 1:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.01, high= 0.05)
        elif teste == 2:
            self.ytm_init = self.ytm_init + np.random.uniform(low=0.05, high= 0.1)

    def risco_sell(self):
        teste = np.random.randint(low=0, high= 3)
        if teste == 0:
            self.ytm_init = self.ytm_init - np.random.uniform(low=0, high= self.ytm_init*0.1)
        elif teste == 1:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.1, high= self.ytm_init*0.2)
        elif teste == 2:
            self.ytm_init = self.ytm_init - np.random.uniform(low=self.ytm_init*0.2, high= self.ytm_init*0.3)
        




In [82]:
def create_bonds():
    data = []
    for i in range(len(cdi["CDI Futuro"])):
        bond = bonds()
        bond.risco_buy()
        title = f"Title {i+1}"
        pv = round(bond.pv, 2)
        ytm = round(bond.ytm_init*100, 2)
        vencimento = (bond.vencimento)
        cupom = (bond.coupon)
        effec = (bond.efecdu)
        convex = (bond.convex)
        data.append({"Nome": title, "Valor Presente": pv, "YTM" : ytm, "Vencimento" : vencimento, "Taxa de Cupom" : cupom, "Duration Efetiva": effec, "Convexidade":convex})
    df = pd.DataFrame(data)
    df.set_index("Nome", inplace=True)
    return df.sort_values(by="Vencimento", ascending=True)

create_bonds()

,Valor Presente,YTM,Vencimento,Taxa de Cupom,Duration Efetiva,Convexidade
Nome,,,,,,
Title 2,1292.00,8.86,1,37,0.901840,1.278278
Title 6,1339.70,6.80,1,42,0.895472,1.265937
Title 1,1452.84,7.34,2,32,1.629561,3.692178
Title 8,1544.38,7.52,2,37,1.603197,3.607635
Title 9,1178.22,7.31,2,17,1.733232,4.024626
Title 3,1591.44,12.61,4,25,2.905240,11.272382
Title 4,2182.96,13.56,5,37,3.214074,14.327642
Title 5,1003.17,9.32,6,9,4.565754,26.124744
Title 7,1185.90,9.39,6,13,4.303553,23.988839
